# Guided Project
# Creating Relations in SQLite

As a quick refresher, the Academy Awards, also known as the Oscars, is an annual awards ceremony hosted to recognize the achievements in the film industry. There are many different awards categories and the members of the academy vote every year to decide which artist or film should get the award. Each row in our data represents a nomination for an award. Recall that our database file, `nominations.db`, contains just the `nominations` table. This table has the following schema:

* `Year` - the year of the awards ceremony, integer type.
* `Category` - the category of award the nominee was nominated for, text type.
* `Nominee` - the person nominated for the award, text type.
* `Movie` - the movie the nominee participated in, text type.
* `Character` - the name of the character the nominee played, text type.
* `Won` - if this nominee won the award, integer type.

Let's now set up our enviroment and spend some time getting familiar with the data before we start normalizing it.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('nominations.db')

In [3]:
pd.read_sql('select * from nominations limit 10', conn)

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston
5,2010,Actor -- Supporting Role,Christian Bale,1,The Fighter,Dicky Eklund
6,2010,Actor -- Supporting Role,John Hawkes,0,Winter's Bone,Teardrop
7,2010,Actor -- Supporting Role,Jeremy Renner,0,The Town,James Coughlin
8,2010,Actor -- Supporting Role,Mark Ruffalo,0,The Kids Are All Right,Paul
9,2010,Actor -- Supporting Role,Geoffrey Rush,0,The King's Speech,Lionel Logue


In [4]:
schema = conn.cursor().execute('pragma table_info(nominations)').fetchall()
schema

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

In [5]:
first_ten = conn.cursor().execute('select * from nominations limit 10').fetchall()
#first_ten

In [6]:
for sc in schema:
    print(sc)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [7]:
for row in first_ten:
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


Let's now add information on the host for each awards ceremony. Instead of adding a `Host` column to the `nominations` table and having lots of redundant data, we'll create **a separate table called `ceremonies`** which contains data specific to the ceremony itself. <br>

Let's create a `ceremonies` table that contains the `Year` and `Host` for each ceremony and then set up a **one-to-many relationship** between `ceremonies` and `nominations`. In this screen, we'll focus on creating the ceremonies table and inserting the data we need and in the next guided step, we'll focus on setting up the one-to-many relationship.

The `ceremonies` table will contain 3 fields:

* `id` - unique identifier for each row, integer type.
* `Year` - the year of the awards ceremony, integer type.
* `Host` - the host of the awards ceremony, text type.

Before we can create and insert into the `ceremonies` table, we need to look up the host for each ceremony from 2000 to 2010. While we could represent each row as a tuple and write a SQL query with an `INSERT` statement to add each row to the `ceremonies` table, this is incredibly cumbersome. <br>

The Python sqlite3 library comes with an [`executemany` method](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany) that let's us easily mass insert records into a table. The `executemany` method requires the records we want to insert to be represented as **a list of tuples**. We then just need to write a single `INSERT` query with **placeholder** elements and specify that we want the list of tuples to be dropped into the query. <br>

Let's first create the list of tuples representing the data we want inserted and then we'll walk through the placeholder query we need to write. We'll skip over creating the `ceremonies` table for now since we've explored how to create a table earlier in the course.

[Wikipedia](https://en.wikipedia.org/wiki/List_of_Academy_Awards_ceremonies#Ceremonies) lists the host(s) for each awards ceremony. Even though the 2010 ceremony had 2 hosts, we selected just the first host, `Steve Martin`, to keep things simple. Here's what the list of tuples looks like:

```python
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
```

We then need to write the `INSERT` query with placeholder values. Instead of having specific values in the query string, we use a question mark (`?`) to act as a placeholder in the `values` section of the query:

```python
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
```

Since the **placeholder elements** (`?`) will be replaced by the values in years_hosts, you need to make sure the number of question marks matches the length of each tuple in years_hosts. Since each tuple has 2 elements, we need to have 2 question marks as the placeholder elements. We don't need to specify values for the id column since it's a primary key column. **When inserting values, recall that SQLite automatically creates a unique primary key for each row**. <br>

We then call the `executemany` method and pass in `insert_query` as the first parameter and `years_hosts` as the second parameter:

```sql
conn.executemany(insert_query, years_hosts)
```

Python will **iterate through years_hosts** and **replace the placeholder elements with the values in years_hosts** to generate and execute the following query:

```python
INSERT INTO ceremonies (Year, Host) VALUES
(2010, "Steve Martin"),
(2009, "Hugh Jackman"),
(2008, "Jon Stewart"),
(2007, "Ellen DeGeneres"),
(2006, "Jon Stewart"),
(2005, "Chris Rock"),
(2004, "Billy Crystal"),
(2003, "Steve Martin"),
(2002, "Whoopi Goldberg"),
(2001, "Steve Martin"),
(2000, "Billy Crystal")
;
```

Let's now create the ceremonies table and populate the table with the data on ceremonies.

In [8]:
conn.cursor().execute('drop table if exists ceremonies'); conn.commit()

In [9]:
create_table_query = '''
        create table ceremonies (
                id integer primary key,
                year integer,
                host varchar
        )
        
'''

conn.cursor().execute(create_table_query)
conn.commit()

* Create the `ceremonies` table with the following schema:
  * `id`: integer, primary key.
  * `Year`: integer.
  * `Host`: text.
* Create the list of tuples, `years_hosts`, that contains the values for the rows we want to insert into the `ceremonies` table.

* Use the Connection method `executemany` to insert the values into the `ceremonies` table.

* To verify that the `ceremonies` table was created and populated correctly, run the following queries:
  * Return the first 10 rows using the `SELECT` and `LIMIT` statements.
  * Return the schema using `pragma table_info()`.
  

In [10]:
years_hosts = [(2010, "Steve Martin"),
                (2009, "Hugh Jackman"),
                (2008, "Jon Stewart"),
                (2007, "Ellen DeGeneres"),
                (2006, "Jon Stewart"),
                (2005, "Chris Rock"),
                (2004, "Billy Crystal"),
                (2003, "Steve Martin"),
                (2002, "Whoopi Goldberg"),
                (2001, "Steve Martin"),
                (2000, "Billy Crystal")]

In [11]:
insert_query = "insert into ceremonies (Year, Host) VALUES (?,?);"

In [12]:
conn.executemany(insert_query, years_hosts)
conn.commit()

In [13]:
pd.read_sql('select * from ceremonies limit 5', conn)

,id,year,host
0,1,2010,Steve Martin
1,2,2009,Hugh Jackman
2,3,2008,Jon Stewart
3,4,2007,Ellen DeGeneres
4,5,2006,Jon Stewart


In [14]:
pd.read_sql('pragma table_info(ceremonies)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,year,integer,0,None,0
2,2,host,varchar,0,None,0


Since we'll be creating relations using **foreign keys**, we need to turn on foreign key **constraints**. By default, if you insert a row into a table that contains one or multiple foreign key columns, the record will be successfully inserted even if the foreign key reference is incorrect. <br>

For example, since the `ceremonies` table only contains the `id` values `1` to `10`, inserting a row into `nominations` while specifying that the `ceremony_id` value be `11` will work and no error will be returned. This is problematic because if we try to actually join that row with the `ceremonies` table, the results set will be empty since the `id` value `11` doesn't map to any row in the `ceremonies` table. To prevent us from inserting rows with nonexisting foreign key values, we need to turn on foreign key constraints by running the following query:

```
PRAGMA foreign_keys = ON;
```

### **The above query needs to be run every time you connect to a database where you'll be inserting foreign keys.** 
Whenever you try inserting a row into a table containing foreign key(s), SQLite will query the linked table to make sure that foreign key value exists. If it does, the transaction will continue as expected. If it doesn't, then an error will be returned and the transaction won't go through.

In [15]:
conn.cursor().execute('pragma foreign_keys = ON'); conn.commit()

The next step is to remove the `Year` column from `nominations` and add a new column, `ceremony_id`, that contains the foreign key reference to the id column in the `ceremonies` table. Unfortunately, we can't remove columns from an existing table in SQLite or change its schema. [The goal of SQLite](https://en.wikipedia.org/wiki/SQLite#History) is to create an incredibly lightweight, open source database that contains a common, but reduced, set of features. While this has allowed SQLite to [become the most popular database in the world](https://www.sqlite.org/mostdeployed.html), SQLite doesn't have the ability to heavily modify an existing table to keep the code base lightweight.<br>

The only alterations we can make to an existing table are renaming it or adding a new column. This means that we can't just remove the `Year` column from `nominations` and add the `ceremony_id` column. We need to instead:

* create a new table nominations_two with the schema we want,
* populate nominations_two with the records we want,
* delete the original nominations table,
* rename nominations_two to nominations.

For nominations_two, we want the following schema:

* `id`: primary key, integer,
* `category`: text,
* `nominee`: text,
* `movie`: text,
* `character`: text,
* `won`: integer,
* `ceremony_id`: foreign key reference to `id` column from `ceremonies`.

First, we need to select all the records from the original `nominations` table with the columns we want and use an `INNER JOIN` to add the `id` field from `ceremonies` for each row:

```sql
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
;
```

Then we can write the placeholder insert query we need to insert these records into `nominations_two`. Let's create and populate the `nominations_two` table in this screen and we'll work through the rest in the next screen.

* Write and run the query to create the `nominations_two` table with the schema specified above.
* Write and run the query we discussed above that returns the records from the `nominations` table and assign the results set to `joined_nominations`.
* Write a placeholder insert query that can insert values into `nominations_two` and assign this query to `insert_nominations_two`. Make sure the ordering of the columns matches the ordering from `joined_nominations`.
* Use the Connection method `executemany` to insert the records in `joined_nominations` using the placeholder insert query `insert_nominations_two`.
* Verify your work by returning the first 5 rows from `nominations_two`.

In [49]:
conn.cursor().execute('drop table if exists nominations_two'); conn.commit()

In [50]:
create_nomination_two = '''
        create table nominations_two 
            (
            id integer primary key, 
            category text, 
            nominee text, 
            movie text, 
            character text, 
            won integer,
            ceremony_id integer,
            foreign key(ceremony_id) references ceremonies(id)
            );
'''

In [51]:
conn.cursor().execute(create_nomination_two); conn.commit()

In [52]:
join_nominations = '''
        select N.category, N.nominee, N.movie, N.character,
                N.won, C.id as ceremony_id
        from nominations N
        inner join ceremonies C
        on N.year == C.year;
'''

joined_nominations = conn.cursor().execute(join_nominations).fetchall()
#joined_nominations

In [53]:
pd.read_sql(join_nominations, conn).head()

,Category,Nominee,Movie,Character,Won,ceremony_id
0,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1


In [54]:
insert_nominations_two = '''
        insert into nominations_two (Category, Nominee, Movie, Character,
                                    Won, ceremony_id)
                                    values (?, ?, ?, ?, ?, ?);
'''
conn.cursor().executemany(insert_nominations_two,\
                         joined_nominations)

In [55]:
pd.read_sql('select * from nominations_two limit 5', conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1


We now need to delete the `nominations` table since we'll be using the `nominations_two` table moving forward. We can use the `DROP TABLE` statement to drop the original `nominations` table. <br>

Once we drop this table, we can use the `ALTER TABLE` statement to rename `nominations_two` to `nominations`. Here's what the syntax looks like for that statement:

```
ALTER TABLE [current_table_name]
RENAME TO [future_table_name]
```

In [56]:
conn.cursor().execute('drop table nominations')
conn.cursor().execute('alter table nominations_two\
                      rename to nominations')
conn.commit()

In [57]:
pd.read_sql('select * from nominations limit 5', conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1


In the [Database Normalization and Relations mission](https://www.dataquest.io/mission/181/database-normalization-and-relations), we learned how to query the tables involved in the many-to-many relation between movies and actors.

Here's a preview of what those tables look like:
![](img/1.png)

Creating a join table is no different than creating a regular one. To create the `movies_actors` join table we need to declare both of the foreign key references when specifying the schema:

```sql
CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id),
actor_id INTEGER REFERENCES actors(id)
);
```

In this screen, you'll create the 3 tables we need.

#### instructions

* Create the 3 tables we need to model the relationship between movies and actors. You need to create the movies and actors tables before creating the movies_actors table for the foreign key references to work.
* Create the movies table using the following schema:
  * id: primary key, integer type.
  * movie: movie name, text type.
* Create the actors table using the following schema:
  * id: primary key, integer type.
  * actor: actor's full name, text type.
* Create the movies_actors join table using the following schema:
  * id: primary key, integer type.
  * movie_id: foreign key reference to movies.id column.
  * actor_id: foreign key reference to actors.id column.

In [62]:
create_movies = '''create table movies(
        id integer primary key,
        movie text
);
'''

create_actors = '''create table actors(
        id integer primary key,
        actor text
);
'''

create_movies_actors = '''create table movies_actors(
        id integer primary key,
        movie_id integer,
        actor_id integer,
        foreign key(movie_id) references movies(id),
        foreign key(actor_id) references actors(id)
);
'''

In [64]:
conn.cursor().execute(create_movies)
conn.cursor().execute(create_actors)
conn.cursor().execute(create_movies_actors)
conn.commit()

# Populating 'movies' & 'actors' tables

In [67]:
pop_movies = 'insert into movies (movie) select distinct movie from nominations'
pop_actors = 'insert into actors (actor) select distinct nominee from nominations'
conn.execute(pop_movies)
conn.execute(pop_actors)

print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


In [69]:
pd.read_sql('select * from movies limit 5', conn)

,id,movie
0,1,Biutiful
1,2,True Grit
2,3,The Social Network
3,4,The King's Speech
4,5,127 Hours


In [70]:
pd.read_sql('select * from actors limit 5', conn)

,id,actor
0,1,Javier Bardem
1,2,Jeff Bridges
2,3,Jesse Eisenberg
3,4,Colin Firth
4,5,James Franco


# Populating 'movies_actors' join table

In [68]:
pd.read_sql('select * from movies_actors limit 5', conn)

,id,movie_id,actor_id


In [72]:
pairs_query = 'select movie, nominee from nominations;'
movie_actor_pairs = conn.execute(pairs_query).fetchall()

In [75]:
join_table_insert = '''
        insert into movies_actors (movie_id, actor_id)
                    values((select id from movies where movie == ?),
                             (select id from actors where actor == ?)
                            );
'''

In [76]:
conn.executemany(join_table_insert, movie_actor_pairs)

print(conn.execute('select * from movies_actors limit 5').fetchall())

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]
